In [46]:
import requests

In [47]:
# Target ESCO concept URI — for example: ICT field in ISCED-F
uri = "http://data.europa.eu/esco/isced-f/054"
language = "es"

# ESCO API endpoint for SKOS concept resources
url = "https://ec.europa.eu/esco/api/resource/concept"

# Query parameters for the GET request
params = {
    "uri": uri,
    "language": language
}

# Headers to request JSON data
headers = {
    "Accept": "application/json"
}

# Send GET request to the ESCO API
response = requests.get(url, params=params, headers=headers)

# Check for a successful response
if response.status_code == 200:
    data = response.json()
    
    narrower_concepts = data.get("narrowerConcept", {}).get("resourceList", [])



In [ ]:
# Target ESCO concept URI — for example: ICT field in ISCED-F
uri = "http://data.europa.eu/esco/isced-f/054"
language = "es"

# ESCO API endpoint for SKOS concept resources
url = "https://ec.europa.eu/esco/api/resource/skill"


# Query parameters for the GET request
params = {
    "uri": uri,
    "language": language
}

# Headers to request JSON data
headers = {
    "Accept": "application/json"
}

# Send GET request to the ESCO API
response = requests.get(url, params=params, headers=headers)

# Check for a successful response
if response.status_code == 200:
    data = response.json()
    
    narrower_concepts = data.get("narrowerConcept", {}).get("resourceList", [])

In [48]:
response.json()

{'timestamp': '2025-07-01 22:26:19',
 'status': 404,
 'error': 'Not Found',
 'message': '',
 'path': '/esco/api/resource/concept/skill'}

In [35]:
# Check for a successful response
if response.status_code == 200:
    data = response.json()
    
    narrower_concepts = data.get("_links").get("narrowerConcept")
    

In [42]:
data.get("_links").get("narrowerConcept")

[{'href': 'https://ec.europa.eu/esco/api/resource/concept?uri=http://data.europa.eu/esco/isced-f/0540&language=en',
  'uri': 'http://data.europa.eu/esco/isced-f/0540',
  'title': 'mathematics and statistics not further defined'},
 {'href': 'https://ec.europa.eu/esco/api/resource/concept?uri=http://data.europa.eu/esco/isced-f/0542&language=en',
  'uri': 'http://data.europa.eu/esco/isced-f/0542',
  'title': 'statistics'},
 {'href': 'https://ec.europa.eu/esco/api/resource/concept?uri=http://data.europa.eu/esco/isced-f/0541&language=en',
  'uri': 'http://data.europa.eu/esco/isced-f/0541',
  'title': 'mathematics'}]

In [43]:
data

{'className': 'Concept',
 'classId': 'http://www.w3.org/2004/02/skos/core#Concept',
 'uri': 'http://data.europa.eu/esco/isced-f/054',
 'title': 'mathematics and statistics',
 'preferredLabel': {'no': 'matematikk og statistikk',
  'de': 'Mathematik und Statistik',
  'fi': 'matematiikka ja tilastotiede',
  'pt': 'matemática e estatística',
  'bg': 'математика и статистика',
  'lt': 'matematika ir statistika',
  'lv': 'matemātika un statistika',
  'hr': 'matematika i statistika',
  'fr': 'mathématiques et statistiques',
  'hu': 'matematika és statisztika',
  'sk': 'matematika a štatistika',
  'sl': 'matematika in statistika',
  'ga': 'matamaitic agus staidreamh',
  'sv': 'matematik och statistik',
  'mt': 'il-matematika u l-istatistika',
  'el': 'μαθηματικά και στατιστική',
  'en': 'mathematics and statistics',
  'is': 'stærðfræði og tölfræði',
  'it': 'matematica e statistica',
  'es': 'matemáticas y estadística',
  'et': 'matemaatika ja statistika',
  'cs': 'matematika a statistika',
  